In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from food.clipmodel import *

# !nbdev_build_lib

In [3]:
#https://data.world/kgarrett/whats-on-the-menu/workspace/query?queryid=5d1d1edb-5e6c-4bc7-bf9e-b128f846b959
# pd.read_csv('food.csv').to_sql('foods',engine,index=False,if_exists='append')

In [4]:
# from tendo import singleton
# me = singleton.SingleInstance()

import os
os.environ["HF_DATASETS_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from food.tools import *
from food.psql import *
from food.paths import *
from PIL import Image
from food.clipmodel import image2clip
import custom_pandas as cpd
from tqdm import tqdm

In [51]:
query = """select id,product_name,keywords,ingredients_text,categories,food_groups
             from foods_big
            where id = 24599 """

In [52]:
total = engine.execute(f'select count(*) from ({query}) a').one()

In [53]:
bs = 1
pd_iter = cpd.read_sql_query(query, engine, chunksize=bs, index_col='id')

In [54]:
inp = next(pd_iter)

In [55]:
inp

,product_name,keywords,ingredients_text,categories,food_groups
id,,,,,
24599,Gummy Worms,"co, candy, worm, inc, chri, gummy","Corn syrup, sucrose, gelatin, citric acid, app...",NaN,NaN


In [56]:
text =inp.fillna("")
for c in text.columns: text[c] = text[c].str.replace('NaN','')
text = text['product_name']+ '. ' + text['food_groups']+ '. '+ text['categories']+ '. ' + text['ingredients_text']+'. ' + text['keywords'] 

In [57]:
text.tolist()[0][:278]

'Gummy Worms. . . Corn syrup, sucrose, gelatin, citric acid, apple juice concentrate, sodium citate, coconut oil, natural and artificial flavors, carnauba wax, carnuba wax, ascorbic acid (vitamin c) red 40. yellow 5, and blue 1.. co, candy, worm, inc, chri, gummy'

In [65]:
clip = text2clip(text.tolist()[0][:250]).numpy().tolist()

In [12]:
for inp in tqdm(pd_iter, desc="clip food inference", total=total[0] // bs):  
    text =inp.fillna("")
    for c in text.columns: text[c] = text[c].str.replace('NaN','')
    text = text['product_name']+ '. ' + text['food_groups']+ '. ' + text['keywords'] + '. ' + text['ingredients_text'] + '. '+ text['categories'] 
    clip = text2clip(text.tolist()[0]).numpy().tolist()
    inp['clip'] = [clip]
    insert_ignore(inp,'foods_big',update=True,update_cols=['clip'],unique_cols=['id'])


clip food inference: 0it [00:00, ?it/s]


In [3]:
1

1